#### Copyright 2019 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Deep Neural Networks in Tensorflow

In this lesson, we will train a deep neural network to interpret sign language from images of hands forming sign language shapes.

The [dataset](https://www.kaggle.com/datamunge/sign-language-mnist) contains images of sign language symbols. The 'j' and 'z' are not included in the dataset since these letters require not just a hand form, but also a motion.

## Acquire the Data

Load the zipped sign language folder into Colab. The file should be named `sign-language-mnist.zip`.

After the file is uploaded, you can unzip the file, as below.

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('sign-language-mnist.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

The unzipped data contains many different files. There is an overview image of American Sign Language (ASL), a few example images, and two CSV files. The dataset is already split into testing and training data.

## Load and Examine the Training Data

Let's load the data and describe it. As we can see there are 27,455 rows of data. Each row starts with a label signifying the letter in the image, and 784 pixels that contain the data for a 28x28 image.

The label contains values 0.0 through 24.0 and the pixels contain data between 0.0 and 255.0.

In [0]:
import pandas as pd

train_df = pd.read_csv('sign_mnist_train.csv')
train_df.describe()

Let's take a look at the distribution of the letters.

In [0]:
train_df.groupby('label')['label'].count()

We can see that letter 9 ('j') and letter 25 ('z') are missing, as expected. There seems to be a relatively even number of training samples for each letter.

Just to get an idea of the images that we are working with, let's take a look at a sample.

In [0]:
import matplotlib.pyplot as plt

image_to_view = 100

plt.imshow(
  train_df.iloc[image_to_view][train_df.columns.values[1:]].values.reshape(28, 28),
  cmap='gray')
plt.show()

## Load and Examine the Test Data

We can see that there are 7,172 test records, which is roughly a 20% sample.

In [0]:
test_df = pd.read_csv('sign_mnist_test.csv')
test_df.describe()

Let's see the distribution of the labels in the test data. The test labels are also pretty evenly distributed.

In [0]:
test_df.groupby('label')['label'].count()

## Data Preparation

Since our pixel values range from 0 through 255, their values might not work well in a neural network. It is best to scale down the data if possible. For this pixel data scaling is as easy as dividing by 255. We should also convert our label to an integer value.

In [0]:
train_df[train_df.columns.values[0]] = train_df[train_df.columns.values[0]].astype(int)
train_df[train_df.columns.values[1:]] = train_df[train_df.columns.values[1:]]/255.0
train_df.describe()

We need to scale the test data too, and convert it to the right type.

In [0]:
test_df[test_df.columns.values[0]] = test_df[test_df.columns.values[0]].astype(int)
test_df[test_df.columns.values[1:]] = test_df[test_df.columns.values[1:]]/255.0
test_df.head()

## Build and Train the Model

We now have some data ready to train and test our model. Let's begin by creating feature columns.

In [0]:
from tensorflow.feature_column import numeric_column

pixel_features = []

for column_name in train_df.columns[1:]:
  pixel_features.append(numeric_column(column_name))

len(pixel_features)

Creating a deep neural network classifier is as simple as creating an instance of a `DNNClassifier` class, declaring the number of classes, and choosing the number of hidden units.

In [0]:
from tensorflow.estimator import DNNClassifier

classifier = DNNClassifier(feature_columns=pixel_features,
                           hidden_units=[392, 196], n_classes=26)

We can now create an input function to feed the data to the model for training. The input function will create a dataset containing the feature and label columns. We'll shuffle the dataset, process it in batches, and repeat it.

In [0]:
import tensorflow as tf

from tensorflow.data import Dataset

def training_input():
  features = {}
  for i in range(1, len(pixel_features)+1):
    column_name = 'pixel'+str(i)
    features[column_name] = train_df[column_name]
 
  labels = train_df['label']

  training_ds = Dataset.from_tensor_slices((features, labels))
  training_ds = training_ds.shuffle(buffer_size=10000)
  training_ds = training_ds.batch(100)
  training_ds = training_ds.repeat(5)

  return training_ds

classifier.train(training_input)

## Test the Model

We can now create an input function to feed our testing data into the model to make predictions. We'll use the model to make predictions and save them in a list.

In [0]:
def testing_input():
  features = {}
  for i in range(1, len(pixel_features) + 1):
    column_name = 'pixel' + str(i)
    features[column_name] = test_df[column_name]
  return Dataset.from_tensor_slices((features)).batch(1)

predictions = list(classifier.predict(testing_input))

We can now take a look at one of the predictions to see what it looks like.

In [0]:
predictions[0]

We see the predicted class, as well as the logits and probabilities for the class.

Let's look at the actual label for our first bit of test data.

In [0]:
test_df.iloc[0]['label']

We can also see if the image looks like the symbol we predicted.

In [0]:
import matplotlib.pyplot as plt

image_to_view = 0

plt.imshow(
  test_df.iloc[image_to_view][test_df.columns.values[1:]].values.reshape(28, 28),
  cmap='gray')
plt.show()

We should now extract all of the class IDs from our predictions and calculate the F1 score.

In [0]:
predicted_classes = [p['class_ids'][0] for p in predictions]
predicted_classes

In [0]:
from sklearn.metrics import f1_score

f1_score(test_df['label'], predicted_classes, average='micro')

# Exercises

## Exercise 1

The F1 score for this model is pretty poor. Check out the [documentation for the DNNClassifier](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier) and see what parameters you can change.

Experiment with different numbers and widths of hidden layers. Consider trying different optimizers and activation functions.

Record the parameters that you set and your F1 score for those parameters below. Try to get an F1 above 0.5.

### Student Solution

In [0]:
### Run 1
# Batch Size: 100
# Data Set Repeats: 5
# Layers: [392, 196]
# Activation Function: relu
# Loss Reduction: losses.Reduction.SUM
# Optimizer: Adagrad
# F1 Score: ???




### Run 2
# Batch Size: 
# Data Set Repeats: 
# Layers: []
# Activation Function: 
# Loss Reduction: 
# Optimizer: 
# F1 Score: 

# ...

### Answer Key

**Solution**

In [0]:
# TODO